In [1]:
import os

os.environ['YOLO_VERBOSE'] = 'False'

In [12]:
import torch
from torch import nn
import copy

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO('tsr.pt').cuda()

In [4]:
from fine_grained import fine_grained_prune

In [6]:
with torch.inference_mode():
    pruner = fine_grained_prune.FineGrainedPruner(model,0.2)

In [7]:
pruner.apply(model)

In [10]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
) -> float:
  
  metrics = model.val(data = 'road_sign_data.yaml', split = 'val')
  acc = metrics.results_dict['metrics/mAP50(B)']

  return acc

In [11]:
def fine_tune(model: nn.Module):
  model.train(data='road_sign_data.yaml', epochs=10, imgsz=640,device = '0')

In [13]:
num_finetune_epochs = 3

best_sparse_checkpoint = dict()
best_sparse_accuracy = 0
print(f'Finetuning Fine-grained Pruned Sparse Model')
for epoch in range(num_finetune_epochs):
    # At the end of each train iteration, we have to apply the pruning mask
    #    to keep the model sparse during the training
    # model.train(data="coco8.yaml", epochs=1, imgsz=640,device = '0')
    fine_tune(model)
    with torch.inference_mode():
        pruner.apply(model.to('cuda'))
    accuracy = evaluate(model)
    is_best = accuracy > best_sparse_accuracy
    if is_best:
        best_sparse_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
        best_sparse_accuracy = accuracy
    print(f'    Epoch {epoch+1} Sparse Accuracy {accuracy:.2f}% / Best Sparse Accuracy: {best_sparse_accuracy:.2f}%')

Finetuning Fine-grained Pruned Sparse Model


/home/dell/Yolov8_Optimization/venv/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
